In [ ]:
# import pandas as pd
# import config
# import covariance
# import utils
# from importlib import reload

# reload(covariance)
# reload(config)
# reload(utils)

# for file_key in file_keys:
#     df = utils.read_s3_file(file_key).set_index("date")
#     df.index = pd.to_datetime(df.index)
#     msfe_list = covariance.forecast_cov_matrices(df, config.DATES, config.L)
#     msfe_df = pd.concat(msfe_list)
#     print(f"Mean MSFE for {file_key}: {msfe_df.mean()}")
#     original_filename = file_key.split("/")[-1]
#     new_filename = original_filename.replace(".csv", "_forecast_error.csv")
#     output_file_key = f"output/{new_filename}"
#     utils.write_s3_file(msfe_df, output_file_key)

In [9]:
import sagemaker
from sagemaker.processing import ScriptProcessor

In [10]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Define the script processor
script_processor = ScriptProcessor(
    image_uri="382416733822.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3",
    instance_count=1,
    instance_type="ml.t3.medium",
)

script_processor.run(code="forecast.py")

ValueError: An AWS IAM role is required to create a Processing job.

In [7]:
# Define the script processor
script_processor = ScriptProcessor(
    image_uri="382416733822.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3",  # Built-in SageMaker image URI
    role=role,
    instance_count=1,
    instance_type="ml.t3.medium",
)

script_processor.run(code="forecast.py")

In [ ]:
import sagemaker
from sagemaker.processing import ScriptProcessor

# Define SageMaker session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Define the script processor
script_processor = ScriptProcessor(
    image_uri="382416733822.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3",  # Built-in SageMaker image URI
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
)

# Run the processing job
script_processor.run(code="forecast_covariances.py")

In [ ]:
# Parameters
dates = pd.date_range("19900101", "20240401", freq="MS")
L = 120

# Load data
df = utils.read_s3_file("clean/btm.csv").set_index("date")
df.index = pd.to_datetime(df.index)

# Forecast covariance matrices
msfe_list = covariance.forecast_cov_matrices(df, dates, L)

# # Combine results into a single DataFrame
msfe_df = pd.concat(msfe_list)
print(msfe_df)

# # Print the mean of MSFE values
print(msfe_df.mean())

In [ ]:
file_keys = utils.list_s3_files(config.BUCKET_NAME, "clean/")
file_keys = file_keys[2:]

In [ ]:
for file_key in file_keys:
    df = utils.read_s3_file(file_key).set_index("date")
    df.index = pd.to_datetime(df.index)
    msfe_list = covariance.forecast_cov_matrices(df, config.DATES, config.L)
    msfe_df = pd.concat(msfe_list)
    print(f"Mean MSFE for {file_key}: {msfe_df.mean()}")
    original_filename = file_key.split("/")[-1]
    new_filename = original_filename.replace(".csv", "_forecast_error.csv")
    output_file_key = f"output/{new_filename}"
    utils.write_s3_file(msfe_df, output_file_key)